## BENDERS DECOMPOSITION METHOD
### Prepared by Bikey SERANILLA
#### Problem Model

$$\begin{align}\min\ & 150x_1 + 230x_2 + 260x_3 \\
& -\frac{1}{3}(170w_{11} - 238y_{11} + 150w_{21} - 210y_{21} + 36w_{31} + 10w_{41}) \\
& -\frac{1}{3}(170w_{12} - 238y_{12} + 150w_{22} - 210y_{22} + 36w_{32} + 10w_{42}) \\
& -\frac{1}{3}(170w_{13} - 238y_{13} + 150w_{23} - 210y_{23} + 36w_{33} + 10w_{43}) \\
s.t. \\
& x_1 + 2x_2 + x3 \leq 500 \\
& 3x_1 + y_{11} - w_{11} \geq 200 \\
& 3.6x_2 + y_{21} - w_{21} \geq 240 \\
& w_{31} + w_{41} \leq 24x_3 \\
& w_{31} \leq 6000 \\
& 2.5x_1 + y_{12} - w_{12} \geq 200 \\
& 3x_2 + y_{22} - w_{22} \geq 240 \\
& w_{32} + w_{42} \leq 20x_3 \\
& w_{32} \leq 6000 \\
& 2x_1 + y_{13} - w_{13} \geq 200 \\
& 2.4x_2 + y_{23} - w_{23} \geq 240 \\
& w_{33} + w_{43} \leq 16x_3 \\
& w_{33} \leq 6000 \\
& x, y, w \geq 0, y \in \mathbb{Z}\\
\end{align}$$ 


#### Master Model

$$\begin{align}\min\ & 150x_1 + 230x_2 + 260x_3 + V \\
\end{align}$$

$$\begin{align}s.t. \\
& x_1 + 2x_2 + x3 \leq 500 \\
& V \geq 0 \\
&  V \geq (b-F^y)^Tu \\
& V \geq 0, y \in \mathbb{Z}\\
\end{align}$$ 

#### Subroblem Model

$$\begin{align}\min\ & - 170w_{11} + 238y_{11} - 150w_{21} + 210y_{21} - 36w_{31} - 10w_{41}\\
\end{align}$$
$$\begin{align}s.t. \\
& 3x_1 + y_{11} - w_{11} \geq 200 \\
& 3.6x_2 + y_{21} - w_{21} \geq 240 \\
& w_{31} + w_{41} \leq 24x_3 \\
& w_{31} \leq 6000 \\
& x_1, x_2 \geq 0\\
\end{align}$$ 

In [127]:
import gurobipy as gp
from gurobipy import GRB
from numpy.random import randint, binomial
import numpy as np
from math import sqrt
from seaborn import distplot
from math import inf, isclose
import math as math
from itertools import product

In [147]:
# Parameters
T = 2
I = 4
J = 3
IJ = list(product(range(I), range(J)))

c = [[10, 7, 16, 6],
     [0, 0, 0, 0]]
f = [[40, 24, 4],
     [45, 27, 4.5],
     [32, 19.2, 3.2],
     [55, 33, 5.5]]
demand = [3, 5, 7]
b = 120
m = 12

# VARIABLES
x = np.empty((I+1, T+1)).tolist()
y = np.empty((I+1,J+1, T+1)).tolist()

In [157]:
# MASTER PROBLEM
masterProblem = gp.Model()
masterProblem.Params.LogToConsole = 0
masterProblem.ModelSense = GRB.MINIMIZE
x = masterProblem.addVars(I, vtype=GRB.CONTINUOUS, name='x')
V = masterProblem.addVar(lb=0, name='V')
masterProblem.addConstr(gp.quicksum(x[i] for i in range(I)) <= m)
masterProblem.addConstr(gp.quicksum(c[0][i]*x[i] for i in range(I)) >= b)
masterProblem.addConstr(V >= 0)
masterProblem.setObjective(gp.quicksum(c[0][i]*x[i] for i in range(I)) + V, GRB.MINIMIZE)

In [158]:
# SUBPROBLEM
def subProblem(xi, x):
    subProblem = gp.Model()
    subProblem.Params.LogToConsole = 0
    subProblem.ModelSense = GRB.MINIMIZE
    y = subProblem.addVars(I,J, vtype=GRB.CONTINUOUS, name='y')
    subProblem.addConstrs(gp.quicksum(y[i,j] for i in range(I) for j in range(J)) <= x_j[i] for i in range(I))
    subProblem.addConstr(gp.quicksum(y[i,0] for i in range(I)) >= xi)
    subProblem.setObjective(gp.quicksum(f[i][j]*y[i,j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
    subProblem.optimize()
    y_star = subProblem.getAttr('X', y)
    u_star = subProblem.Pi
    ObjVal = subProblem.ObjVal

    return y_star, u_star, ObjVal
    

In [159]:
# L-SHAPED DECOMPOSITION ALGORITHM

K = 20

#Scenarios
S = 10
SCENARIO = np.array([3,3,3,5,5,5,5,7,7,7])
h = np.array([1,1,1,1,1])

#Second-stage Values
ssp_y = np.zeros((S,I*J))   #y_variables
ssp_u = np.zeros((S,5))     #duals / constraints
ssp_o = np.zeros((S))       #objective value

for k in range(K):
    k = k + 1

    #Step 1 : Solve first-stage problem
    FSP = masterProblem.optimize()
    FSP_ObjVal = masterProblem.ObjVal
    x_j = masterProblem.getAttr('X', x)
    # print(x_j)
    
    # Step 2: Solve each second-stage problem
    for s in range(S):
        # ssp = [y_star, w_star, u_star, ObjVal]
        ssp = subProblem(SCENARIO[s], x_j.values())
        ssp_y[s] = ssp[0].values()
        ssp_u[s] = ssp[1]
        ssp_o[s] = ssp[2]
    
    #Step 3: Generate optimality cuts
    #Multi-cut approach
    E = np.dot(ssp_u[:, :-1].T,SCENARIO)
    e = np.dot(ssp_u,h)

    # print(ssp_u)
    # print(e)

#     # SCENARIOS_constraint = np.insert(SCENARIO, SCENARIO.shape[1], 0, axis=1)
#     # E = sum(np.dot(ssp_u[i].T,SCENARIOS_constraint[i]) for i in range(len(SCENARIO)))/S
#     # e = sum(np.dot(ssp_u[i].T,h) for i in range(len(SCENARIO)))/S

    for s in range(S):
        masterProblem.addConstr(V >= e[s] - gp.quicksum(E[i]*x[i] for i in range(I)))

    # Update bounds
    LB = max(LB, FSP_ObjVal)
    UB = min(UB, LB + np.mean(ssp_o))
    bound = UB - LB

    if bound <= ε:
        break

print("Optimal value:", UB)
x_star = masterProblem.getAttr('X', x)
x_star

GurobiError: Unable to retrieve attribute 'X'

In [109]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
from itertools import product

# Parameters
T = 2
I = 4
J = 3
IJ = list(product(range(I), range(J)))

c = [[10, 7, 16, 6],
     [0, 0, 0, 0]]
f = [[40, 24, 4],
     [45, 27, 4.5],
     [32, 19.2, 3.2],
     [55, 33, 5.5]]
demand = [3, 5, 7]
b = 120
m = 12

# MASTER PROBLEM
masterProblem = gp.Model()
masterProblem.Params.LogToConsole = 0
masterProblem.ModelSense = GRB.MINIMIZE
x = masterProblem.addVars(I, vtype=GRB.CONTINUOUS, name='x')
V = masterProblem.addVar(lb=0, name='V')
masterProblem.addConstr(gp.quicksum(x[i] for i in range(I)) >= m)
masterProblem.addConstr(gp.quicksum(c[0][i] * x[i] for i in range(I)) <= b)
masterProblem.setObjective(gp.quicksum(c[0][i] * x[i] for i in range(I)) + V, GRB.MINIMIZE)

# SUBPROBLEM
def subProblem(demand_scenario, x_values):
    subProblem = gp.Model()
    subProblem.Params.LogToConsole = 0
    subProblem.ModelSense = GRB.MINIMIZE
    y = subProblem.addVars(I, J, vtype=GRB.CONTINUOUS, name='y')
    subProblem.addConstrs((gp.quicksum(y[i, j] for j in range(J)) == x_values[i] for i in range(I)), name='c1')
    subProblem.addConstr(gp.quicksum(y[i, 0] for i in range(I)) <= demand_scenario, name='c2')
    subProblem.setObjective(gp.quicksum(f[i][j] * y[i, j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
    subProblem.optimize()

    y_star = subProblem.getAttr('X', y)
    duals = [c.Pi for c in subProblem.getConstrs()]
    ObjVal = subProblem.ObjVal

    return y_star, duals, ObjVal

# L-SHAPED DECOMPOSITION ALGORITHM
K = 10
ε = 0.0001
UB = math.inf
LB = -math.inf
bound = UB - LB

# Scenarios
S = 10
SCENARIO = np.array([3, 3, 3, 5, 5, 5, 5, 7, 7, 7])
h = np.ones(5)

# Second-stage Values
ssp_y = np.zeros((S, I*J))  # y_variables
ssp_u = np.zeros((S, 1 + I))  # duals / constraints
ssp_o = np.zeros((S))  # objective value

for k in range(K):
    # Step 1: Solve first-stage problem
    masterProblem.optimize()
    FSP_ObjVal = masterProblem.ObjVal
    x_j = masterProblem.getAttr('X', x)
    x_values = np.array([x_j[i] for i in range(I)])

    # Step 2: Solve each second-stage problem
    for s in range(S):
        ssp = subProblem(SCENARIO[s], x_j.values())
        ssp_y[s] = ssp[0].values()
        ssp_u[s] = ssp[1]
        ssp_o[s] = ssp[2]
        
    # Step 3: Generate optimality cuts (multi-cut approach)
    for s in range(S):
        dual_constr1, dual_constr2 = ssp_u[s][:2]
        dual_constr3_to_I = ssp_u[s][2:]
        E_s = np.array([dual_constr1 * SCENARIO[s], dual_constr2])
        e_s = dual_constr1 * h[0] + dual_constr2 * h[1] - np.dot(dual_constr3_to_I, h[2:])

        # Add the optimality cut to the master problem
        masterProblem.addConstr(V >= e_s - gp.quicksum(E_s[i] * x[i] for i in range(I)))

    # Update bounds
    expected_second_stage_cost = np.mean(ssp_o)
    LB = max(LB, FSP_ObjVal + expected_second_stage_cost)
    UB = min(UB, FSP_ObjVal + expected_second_stage_cost)
    bound = UB - LB

    if bound <= ε:
        break

print("Optimal value:", LB)


TypeError: float() argument must be a string or a real number, not 'tupledict'

In [123]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
from itertools import product

# Parameters
T = 2
I = 4
J = 3
IJ = list(product(range(I), range(J)))

c = [[10, 7, 16, 6],
     [0, 0, 0, 0]]
f = [[40, 24, 4],
     [45, 27, 4.5],
     [32, 19.2, 3.2],
     [55, 33, 5.5]]
demand = [3, 5, 7]
b = 120
m = 12

# MASTER PROBLEM
masterProblem = gp.Model()
masterProblem.Params.LogToConsole = 0
masterProblem.ModelSense = GRB.MINIMIZE
x = masterProblem.addVars(I, vtype=GRB.CONTINUOUS, name='x')
V = masterProblem.addVar(lb=0, name='V')
masterProblem.addConstr(gp.quicksum(x[i] for i in range(I)) >= m)
masterProblem.addConstr(gp.quicksum(c[0][i] * x[i] for i in range(I)) <= b)
masterProblem.setObjective(gp.quicksum(c[0][i] * x[i] for i in range(I)) + V, GRB.MINIMIZE)

# SUBPROBLEM
def subProblem(demand_scenario, x_values):
    subProblem = gp.Model()
    subProblem.Params.LogToConsole = 0
    subProblem.ModelSense = GRB.MINIMIZE
    y = subProblem.addVars(I, J, vtype=GRB.CONTINUOUS, name='y')
    subProblem.addConstrs((gp.quicksum(y[i, j] for j in range(J)) == x_values[i] for i in range(I)), name='c1')
    subProblem.addConstr(gp.quicksum(y[i, 0] for i in range(I)) <= demand_scenario, name='c2')
    subProblem.setObjective(gp.quicksum(f[i][j] * y[i, j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
    subProblem.optimize()

    y_star = np.array([subProblem.getVarByName(f'y[{i},{j}]').x for i in range(I) for j in range(J)])
    duals = subProblem.getAttr(GRB.Attr.Pi, subProblem.getConstrs())
    ObjVal = subProblem.ObjVal

    return y_star.reshape((I, J)), duals, ObjVal

# L-SHAPED DECOMPOSITION ALGORITHM
K = 10
ε = 0.0001
UB = math.inf
LB = -math.inf
bound = UB - LB

# Scenarios
S = 10
SCENARIO = np.array([3, 3, 3, 5, 5, 5, 5, 7, 7, 7])
h = np.ones(5)

# Second-stage Values
ssp_y = np.zeros((S, I, J))  # y_variables
ssp_u = np.zeros((S, 2 + I))  # duals / constraints
ssp_o = np.zeros((S))  # objective value

for k in range(K):
    # Step 1: Solve first-stage problem
    masterProblem.optimize()
    FSP_ObjVal = masterProblem.ObjVal
    x_j = np.array([x[i].x for i in range(I)])

    # Step 2: Solve each second-stage problem
    for s in range(S):
        ssp_y[s], ssp_u[s], ssp_o[s] = subProblem(SCENARIO[s], x_j)

    # Step 3: Generate optimality cuts (multi-cut approach)
    for s in range(S):
        dual_constr1, dual_constr2 = ssp_u[s][:2]
        dual_constr3_to_I = ssp_u[s][2:]
        h_adjusted = h[:len(dual_constr3_to_I)]
        E_s = np.array([dual_constr1 * SCENARIO[s], dual_constr2])
        e_s = dual_constr1 * h_adjusted[0] + dual_constr2 * h_adjusted[1] - np.dot(dual_constr3_to_I, h_adjusted)

        # Add the optimality cut to the master problem
        masterProblem.addConstr(V >= e_s - gp.quicksum(E_s[i] * x[i] for i in range(I)))

    # Update bounds
    expected_second_stage_cost = np.mean(ssp_o)
    LB = max(LB, FSP_ObjVal + expected_second_stage_cost)
    UB = min(UB, FSP_ObjVal + expected_second_stage_cost)
    bound = UB - LB

    if bound <= ε:
        break

print("Optimal value:", LB)


ValueError: could not broadcast input array from shape (5,) into shape (6,)